In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import LlamaModel, LlamaTokenizerFast, AdamW
from tqdm.auto import tqdm
import os
import torch.nn.parallel
import torch.nn as nn
# CUDA 사용 가능 여부 확인
os.environ["CUDA_VISIBLE_DEVICES"]= "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Tokenizer

In [3]:
# 데이터 로드

data = pd.read_csv('../../data/train.csv')
# 토크나이저 로드
tokenizer = LlamaTokenizerFast.from_pretrained('beomi/llama-2-ko-7b', eos_token='</s>')
# 데이터 포맷팅 및 토크나이징
formatted_data = []
for _, row in tqdm(data.iterrows()):
    for q_col in ['질문_1', '질문_2']:
        for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
            # 질문과 답변 쌍을 </s> token으로 연결
            input_text = row[q_col] + tokenizer.eos_token + row[a_col]
            input_ids = tokenizer.encode(input_text, return_tensors='pt')
            formatted_data.append(input_ids)
print('Done.')

0it [00:00, ?it/s]

Done.


Model Fine-tuning

In [4]:
# 모델 로드
model = LlamaModel.from_pretrained('beomi/llama-2-ko-7b')
model.to(device) # 모델을 GPU단으로 이동

# 모델 학습 하이퍼파라미터(Hyperparameter) 세팅
# 실제 필요에 따라 조정하세요.
CFG = {
    'LR' : 2e-5, # Learning Rate
    'EPOCHS' : 50, # 학습 Epoch
}

# 모델 학습 설정
optimizer = AdamW(model.parameters(), lr=CFG['LR'])
model.train()

# 모델 학습
for epoch in range(CFG['EPOCHS']):
    total_loss = 0
    progress_bar = tqdm(enumerate(formatted_data), total=len(formatted_data))
    for batch_idx, batch in progress_bar:
        # 데이터를 GPU단으로 이동
        batch = batch.to(device)
        outputs = model(batch)
        loss = outputs.logits
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        # 진행률 표시줄에 평균 손실 업데이트
        progress_bar.set_description(f"Epoch {epoch+1} - Avg Loss: {total_loss / (batch_idx+1):.4f}")

    # 에폭의 평균 손실을 출력
    print(f"Epoch {epoch+1}/{CFG['EPOCHS']}, Average Loss: {total_loss / len(formatted_data)}")

# 모델 저장
model.save_pretrained("../../model/hansoldeco-llama2")
tokenizer.save_pretrained("../../model/hansoldeco-llama2")

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

/home/gil/anaconda3/envs/LeeYS/lib/python3.9/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/6440 [00:00<?, ?it/s]

AttributeError: 'BaseModelOutputWithPast' object has no attribute 'logits'

In [5]:
outputs

BaseModelOutputWithPast(last_hidden_state=tensor([[[ 1.3096, -0.9864,  1.4012,  ...,  0.7658,  0.2644,  0.5384],
         [-1.3573,  1.2840,  0.5290,  ...,  0.3049, -0.6223, -0.8751],
         [ 0.5321, -0.0172, -0.7822,  ...,  0.8230, -0.7069, -0.6110],
         ...,
         [ 1.5726, -0.2414, -1.2711,  ..., -1.1356, -3.0137, -0.2928],
         [ 0.5114, -0.5447, -0.2202,  ..., -0.3011, -2.1610, -0.4873],
         [ 0.1173, -0.3073,  0.2729,  ...,  0.2796, -1.4286, -0.2664]]],
       device='cuda:0', grad_fn=<MulBackward0>), past_key_values=((tensor([[[[-4.8459e-01,  2.1348e-01, -4.2549e-02,  ...,  2.6982e-01,
           -2.6845e-01, -4.4454e-02],
          [-5.1720e-01,  4.3094e-01, -8.9779e-02,  ...,  9.7053e-02,
           -2.6391e-02,  1.7526e-01],
          [-4.0099e-01,  2.3401e-01,  3.0506e-01,  ...,  5.5212e-01,
           -4.2855e-01,  5.8312e-01],
          ...,
          [-1.0999e-01,  1.7294e-03,  2.8593e-02,  ..., -6.9454e-02,
            1.2966e-01, -2.8096e-02],
      

In [ ]:
model = LlamaModel.from_pretrained('beomi/llama-2-ko-7b')
model.to(device) # 모델을 GPU단으로 이동

# 모델 학습 하이퍼파라미터(Hyperparameter) 세팅
# 실제 필요에 따라 조정하세요.
CFG = {
    'LR' : 2e-5, # Learning Rate
    'EPOCHS' : 50, # 학습 Epoch
}

# 모델 학습 설정
optimizer = AdamW(model.parameters(), lr=CFG['LR'])
model.train()

# 모델 학습
for epoch in range(CFG['EPOCHS']):
    total_loss = 0
    progress_bar = tqdm(enumerate(formatted_data), total=len(formatted_data))
    for batch_idx, batch in progress_bar:
        # 데이터를 GPU단으로 이동
        batch = batch.to(device)
        outputs = model(batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        # 진행률 표시줄에 평균 손실 업데이트
        progress_bar.set_description(f"Epoch {epoch+1} - Avg Loss: {total_loss / (batch_idx+1):.4f}")

    # 에폭의 평균 손실을 출력
    print(f"Epoch {epoch+1}/{CFG['EPOCHS']}, Average Loss: {total_loss / len(formatted_data)}")

# 모델 저장
model.save_pretrained("../../model/hansoldeco-llama2")
tokenizer.save_pretrained("../../model/hansoldeco-llama2")

In [ ]:
outputs

In [ ]:
# 저장된 Fine-tuned 모델과 토크나이저 불러오기
model_dir = "../../model/hansoldeco-kogpt2"
model = GPT2LMHeadModel.from_pretrained(model_dir)
model.to(device)
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_dir)

# Inference를 위한 test.csv 파일 로드
test = pd.read_csv('../../data/test.csv')

# test.csv의 '질문'에 대한 '답변'을 저장할 리스트
preds = []

# '질문' 컬럼의 각 질문에 대해 답변 생성
for test_question in tqdm(test['질문']):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')

    # 답변 생성
    output_sequences = model.generate(
        input_ids=input_ids.to(device),
        max_length=300,
        temperature=0.9,
        top_k=1,
        top_p=0.9,
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=1
    )

    # 생성된 텍스트(답변) 저장
    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        # 질문과 답변의 사이를 나타내는 eos_token (</s>)를 찾아, 이후부터 출력
        answer_start = full_text.find(tokenizer.eos_token) + len(tokenizer.eos_token)
        answer_only = full_text[answer_start:].strip()
        answer_only = answer_only.replace('\n', ' ')
        preds.append(answer_only)

Submission

In [ ]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(preds)
pred_embeddings.shape
submit = pd.read_csv('../../data/sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()
# 리더보드 제출을 위한 csv파일 생성
submit.to_csv('../../data/baseline_submit.csv', index=False)